In [31]:
import sys
sys.path.append('../utils')

In [32]:
import pipeline
import importlib
import pandas as pd

importlib.reload(pipeline)
from pipeline import extract_session_data, clean_session_data, get_driver_laps, get_manual_finish_dict
from pipeline import assemble_race_dataset, extract_fp2_features, extract_quali_features, assemble_race_dataset_pre_race
from fastf1 import get_session

In [33]:
# Singapore GP 2025 (Round 18)
# This cell was made before the race so it excludes the race finish data 
df_fp2_18 = extract_session_data(2025, "Singapore", "FP2")
df_quali_18 = extract_session_data(2025, "Singapore", "Q")

fp2_clean_18 = clean_session_data(df_fp2_18)
quali_clean_18 = clean_session_data(df_quali_18)

fp2_features_18 = extract_fp2_features(fp2_clean_18)

quali_features_18 = extract_quali_features(quali_clean_18)
                       
race_df_18 = assemble_race_dataset_pre_race(fp2_features_18, quali_features_18)
race_df_18["grand_prix"] = "Singapore"
race_df_18

core           INFO 	Loading data for Singapore Grand Prix - Practice 2 [v3.4.4]
req            INFO 	Using cached data for session_info


req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '43', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Singapore Grand Prix - Qualifyin

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,114.801909,92.060,11,90.202,12,Singapore
1,ALO,110.489846,90.877,13,89.955,10,Singapore
2,ANT,100.927000,92.719,9,89.537,4,Singapore
3,BEA,104.063500,91.711,6,89.868,9,Singapore
4,BOR,112.562500,92.319,8,90.820,16,Singapore
5,COL,104.457000,93.139,9,90.982,18,Singapore
6,GAS,103.171750,92.458,8,91.261,20,Singapore
7,HAD,117.708273,90.846,11,89.846,8,Singapore
8,HAM,108.410571,91.491,7,89.688,6,Singapore
9,HUL,109.689273,92.069,11,90.141,11,Singapore


In [34]:
race_df_18.to_csv("../data/r18_singapore_2025.csv", index=False)

In [35]:
import joblib

model  = joblib.load("../models/logreg_scaled_quali_pre_r18_singapore.pkl")
scaler = joblib.load("../models/scaler_pre_r18_singapore.pkl")

In [36]:
import pandas as pd

df_singapore = pd.read_csv('../data/r18_singapore_2025.csv')
df_singapore.head()

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,114.801909,92.060,11,90.202,12,Singapore
1,ALO,110.489846,90.877,13,89.955,10,Singapore
2,ANT,100.927000,92.719,9,89.537,4,Singapore
3,BEA,104.063500,91.711,6,89.868,9,Singapore
4,BOR,112.562500,92.319,8,90.820,16,Singapore


In [37]:
feature_cols = ['fp2_avg_lap', 'fp2_best_lap', 'fp2_total_laps', 'FastestQualiLap', 'QualiPosition']
X_singapore = df_singapore[feature_cols]

X_singapore_scaled = scaler.transform(X_singapore) 
proba_podium = model.predict_proba(X_singapore_scaled)[:, 1]

results = df_singapore.copy()
results["podium_probability"] = proba_podium

results_sorted = results.sort_values(by="podium_probability", ascending=False)

predicted_podium = results_sorted[["Driver", "podium_probability"]]

In [38]:
predicted_podium

,Driver,podium_probability
15,RUS,0.894233
19,VER,0.892693
14,PIA,0.828713
2,ANT,0.694641
12,NOR,0.537308
8,HAM,0.347043
11,LEC,0.238252
7,HAD,0.163438
3,BEA,0.072940
1,ALO,0.064597


In [39]:
# Singapore GP 2025
predicted_podium.head(3)

,Driver,podium_probability
15,RUS,0.894233
19,VER,0.892693
14,PIA,0.828713
